In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, InputLayer, Input, Flatten, Conv1D, MaxPooling1D
import optuna
import openpyxl
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from keras.losses import SparseCategoricalCrossentropy
from keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
ds = pd.read_csv('intraindata2n.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
x_train = np.array(ds.iloc[:, 5:])
y1_train = np.array(ds[[0]])
y2_train = np.array(ds[[1]])
y3_train = np.array(ds[[2]])
y4_train = np.array(ds[[3]])
y5_train = np.array(ds[[4]])
ds = pd.read_csv('invalidation2n.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
x_val = np.array(ds.iloc[:, 5:])
y1_val = np.array(ds[[0]])
y2_val = np.array(ds[[1]])
y3_val = np.array(ds[[2]])
y4_val = np.array(ds[[3]])
y5_val = np.array(ds[[4]])


In [ ]:
miny3 = np.min(y3_train)
maxy3 = np.max(y3_train)
miny4 = np.min(y4_train)
maxy4 = np.max(y4_train)
miny5 = np.min(y5_train)
maxy5 = np.max(y5_train)
miny31 = np.min(y3_val)
maxy31= np.max(y3_val)
miny41 = np.min(y4_val)
maxy41 = np.max(y4_val)
miny51 = np.min(y5_val)
maxy51 = np.max(y5_val)
y3_train = (y3_train - miny3) / (maxy3 - miny3)
y4_train = (y4_train - miny4) / (maxy4 - miny4)
y5_train = (y5_train - miny5) / (maxy5 - miny5)
y3_val = (y3_val- miny31) / (maxy31 - miny31)
y4_val = (y4_val - miny41) / (maxy41- miny41)
y5_val = (y5_val - miny51) / (maxy51 - miny51)

In [ ]:
input_shape=(200,1)
x_in=Input(shape=input_shape)
conv1=Conv1D(filters=16, kernel_size=3, activation='relu')(x_in)
pool1=MaxPooling1D(strides=2, padding='same')(conv1)
conv2=Conv1D(filters=32, kernel_size=3, activation='relu')(pool1)
pool2=MaxPooling1D(strides=2, padding='same')(conv2)
conv3=Conv1D(filters=64, kernel_size=3, activation='relu')(pool2)
pool3=MaxPooling1D(strides=2, padding='same')(conv3)
conv4=Conv1D(filters=32, kernel_size=3, activation='relu')(pool3)
pool4=MaxPooling1D(strides=2, padding='same')(conv4)
conv5=Conv1D(filters=256, kernel_size=3, activation='relu')(pool4)
pool5=MaxPooling1D(strides=2, padding='same')(conv5)

xx=Flatten()(pool5)
x = Dense(500, activation='relu')(xx)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
out1 = Dense(2, activation='softmax')(x)

x = Dense(800, activation='relu')(xx)
x = Dense(800, activation='relu')(x)
x = Dense(800, activation='relu')(x)
x = Dense(800, activation='relu')(x)
out2 = Dense(3, activation='softmax')(x)
xa = Dense(510, activation='relu')(xx)
xa = Dense(510, activation='relu')(xa)
xa = Dense(510, activation='relu')(xa)
xa = Dense(510, activation='relu')(xa)

out3 = Dense(1, activation='linear')(xa)
xb = Dense(510, activation='relu')(xx)
xb = Dense(510, activation='relu')(xb)
xb = Dense(510, activation='relu')(xb)
xb = Dense(510, activation='relu')(xb)

out4 = Dense(1, activation='linear')(xb)

xc = Dense(510, activation='relu')(xx)
xc = Dense(510, activation='relu')(xc)
xc = Dense(510, activation='relu')(xc)
xc = Dense(510, activation='relu')(xc)

out5 = Dense(1, activation='linear')(xc)
model = Model(inputs=in1, outputs=(out1, out2, out3, out4, out5))



In [ ]:
optimizer=tf.keras.optimizers.Adam(0.00017)
model.compile(optimizer=optimizer, loss=['SparseCategoricalCrossentropy', 'SparseCategoricalCrossentropy', 'mse', 'mse', 'mse'],
              metrics=[['accuracy'], ['accuracy'], ['mse','mae'], ['mse','mae'], ['mse','mae']])
early=EarlyStopping(monitor='val_loss', patience=100, mode='min', restore_best_weights=True)


In [ ]:
history=model.fit(x_train,( y1_train, y2_train, y3_train, y4_train, y5_train), validation_data=(x_val,[y1_val, y2_val, y3_val, y4_val, y5_val]), batch_size=128, epochs=1000, verbose=2,validation_batch_size=128, shuffle=True, callbacks=[early])